#นาย ปพนธ์ ชุณหคล้าย 6210503691

# read csv to hbase


In [ ]:
!hdfs dfs -copyFromLocal /home/hadoop/data_olym/Olympic_Games_Medal_Tally.csv /Olympic_Games_Medal_Tally.csv
!hdfs dfs -copyFromLocal /home/hadoop/data_olym/Olympics_Games.csv /Olympics_Games.csv

2022-08-21 00:11:50,649 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
copyFromLocal: `/Olympic_Games_Medal_Tally.csv': File exists
2022-08-21 00:11:52,907 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
copyFromLocal: `/Olympics_Games.csv': File exists


#Hbase Connection

ใช้คำสั่ง

`hadoop@BigData:~/hbase$ ./bin/hbase thrift start` 

เพื่อทำการสร้าง Connection ไปยัง hbase ผ่าน port 9090

#Create namespace

ใช้คำสั่ง 

`create_namespace "sample_data"`

#Olympic_Games_Medal_Tally

##Create new table

ใช้คำสั่ง 

`create "sample_data:Olympic_Games_Medal_Tally", "data" `

สร้างตารางก่อน import data เข้าไป

In [ ]:
import csv
import happybase
import time

batch_size = 1000
host = "0.0.0.0"
file_path = "/home/hadoop/data_olym/Olympic_Games_Medal_Tally.csv"
namespace = "sample_data"
row_count = 0
start_time = time.time()
table_name = "Olympic_Games_Medal_Tally"
state = 'Olympic_Games_Medal_Tally'

In [ ]:
def connect_to_hbase():
    """ Connect to HBase server.

    This will use the host, namespace, table name, and batch size as defined in
    the global variables above.
    """
    conn = happybase.Connection(host = host,
        table_prefix = namespace,
        table_prefix_separator = ":")
    conn.open()
    table = conn.table(table_name)
    batch = table.batch(batch_size = batch_size)
    return conn, batch


def insert_row(batch, row):
    """ Insert a row into HBase.

    Write the row to the batch. When the batch size is reached, rows will be
    sent to the database.

    Rows have the following schema:
        [ id, keyword, subcategory, type, township, city, zip, council_district,
          opened, closed, status, origin, location ]
    """
    #ใช้ตัวแปร state เช็คค่าว่าเป็นตารางรูปแบบไหน
    if(state == 'Olympic_Games_Medal_Tally'):
      batch.put(row[0], { 
          "data:edition": row[1], 
          "data:edition_id": row[2], 
          "data:year": row[3],
          "data:country": row[4], 
          "data:country_noc": row[5], 
          "data:gold": row[6],
          "data:silver": row[7], 
          "data:bronze": row[8], 
          "data:total": row[9]
          })
      
    if(state == 'Olympics_Games'):
        batch.put(row[0], { 
            "data:edition": row[0],
            "data:edition_url": row[1],
            "data:year": row[2],
            "data:city": row[3], 
            "data:country_flag_url": row[4],
            "data:country_noc": row[5],
            "data:start_date": row[6], 
            "data:end_date": row[7]
            })


def read_csv():
    csvfile = open(file_path, "r")
    csvreader = csv.reader(csvfile)
    return csvreader, csvfile

In [ ]:


# After everything has been defined, run the script.
conn, batch = connect_to_hbase()
print("Connect to HBase. table name: %s, batch size: %i" % (table_name, batch_size))
csvreader, csvfile = read_csv()
print("Connected to file. name: %s" % (file_path))


Connect to HBase. table name: Olympic_Games_Medal_Tally, batch size: 1000
Connected to file. name: /home/hadoop/data_olym/Olympic_Games_Medal_Tally.csv


In [ ]:
try:
    # Loop through the rows. The first row contains column headers, so skip that
    # row. Insert all remaining rows into the database.
    for row in csvreader:
        row_count += 1
        if row_count == 1:
            pass
        else:
            insert_row(batch, row)

    # If there are any leftover rows in the batch, send them now.
    batch.send()
finally:
    # No matter what happens, close the file handle.
    csvfile.close()
    conn.close()

duration = time.time() - start_time
print("Done. row count: %i, duration: %.3f s" % (row_count, duration))

Done. row count: 667, duration: 5.167 s


In [ ]:
 
conn = happybase.Connection(host = host,
        table_prefix = namespace,
        table_prefix_separator = ":")
conn.open()
table = conn.table(table_name)
batch = table.batch(batch_size = batch_size)
    
# Get list of rows.
for k, data in table.scan():
    print(k, data)

b'1000' {b'data:bronze': b'20', b'data:country': b'France', b'data:country_noc': b'FRA', b'data:edition': b'2008 Summer Olympics', b'data:edition_id': b'53', b'data:gold': b'7', b'data:silver': b'16', b'data:total': b'43', b'data:year': b'2008'}
b'1001' {b'data:bronze': b'4', b'data:country': b'Netherlands', b'data:country_noc': b'NED', b'data:edition': b'2008 Summer Olympics', b'data:edition_id': b'53', b'data:gold': b'7', b'data:silver': b'5', b'data:total': b'16', b'data:year': b'2008'}
b'1002' {b'data:bronze': b'11', b'data:country': b'Ukraine', b'data:country_noc': b'UKR', b'data:edition': b'2008 Summer Olympics', b'data:edition_id': b'53', b'data:gold': b'7', b'data:silver': b'4', b'data:total': b'22', b'data:year': b'2008'}
b'1003' {b'data:bronze': b'6', b'data:country': b'Kenya', b'data:country_noc': b'KEN', b'data:edition': b'2008 Summer Olympics', b'data:edition_id': b'53', b'data:gold': b'6', b'data:silver': b'4', b'data:total': b'16', b'data:year': b'2008'}
b'1004' {b'data:

#Olympics_Games

##Create new table

ใช้คำสั่ง 

`create "sample_data:Olympic_Games", "data" `

สร้างตารางก่อน import data เข้าไป

In [ ]:
import csv
import happybase
import time

batch_size = 1000
host = "0.0.0.0"
file_path = "/home/hadoop/data_olym/Olympics_Games.csv"
namespace = "sample_data"
row_count = 0
start_time = time.time()
table_name = "Olympics_Games"
state = 'Olympics_Games'

In [ ]:
# After everything has been defined, run the script.
conn, batch = connect_to_hbase()
print("Connect to HBase. table name: %s, batch size: %i" % (table_name, batch_size))
csvreader, csvfile = read_csv()
print("Connected to file. name: %s" % (file_path))

Connect to HBase. table name: Olympics_Games, batch size: 1000
Connected to file. name: /home/hadoop/data_olym/Olympics_Games.csv


In [ ]:
try:
    # Loop through the rows. The first row contains column headers, so skip that
    # row. Insert all remaining rows into the database.
    for row in csvreader:
        row_count += 1
        if row_count == 1:
            pass
        else:
            insert_row(batch, row)

    # If there are any leftover rows in the batch, send them now.
    batch.send()
finally:
    # No matter what happens, close the file handle.
    csvfile.close()
    conn.close()

duration = time.time() - start_time
print("Done. row count: %i, duration: %.3f s" % (row_count, duration))

Done. row count: 63, duration: 2.763 s


In [ ]:
conn = happybase.Connection(host = host,
        table_prefix = namespace,
        table_prefix_separator = ":")
conn.open()
table = conn.table(table_name)
batch = table.batch(batch_size = batch_size)
    
# Get list of rows.
for k, data in table.scan():
    print(k, data)

b'1896 Summer Olympics' {b'data:bronze': b'na', b'data:city': b'Athina', b'data:country': b'/images/flags/GRE.png', b'data:country_flag_url': b'/images/flags/GRE.png', b'data:country_noc': b'GRE', b'data:edition': b'1896 Summer Olympics', b'data:edition_id': b'1896', b'data:edition_url': b'/editions/1', b'data:end_date': b'1896-04-15', b'data:gold': b'1896-04-06', b'data:silver': b'1896-04-15', b'data:start_date': b'1896-04-06', b'data:total': b'1896-04-06', b'data:year': b'1896'}
b'1900 Summer Olympics' {b'data:bronze': b'na', b'data:city': b'Paris', b'data:country': b'/images/flags/FRA.png', b'data:country_flag_url': b'/images/flags/FRA.png', b'data:country_noc': b'FRA', b'data:edition': b'1900 Summer Olympics', b'data:edition_id': b'1900', b'data:edition_url': b'/editions/2', b'data:end_date': b'na', b'data:gold': b'na', b'data:silver': b'na', b'data:start_date': b'na', b'data:total': b'1900-05-14', b'data:year': b'1900'}
b'1904 Summer Olympics' {b'data:bronze': b'na', b'data:city':

# Your task


from : Olympic_Games_Medal_Tally.csv  

* ,edition,edition_id,year,country,country_noc,gold,silver,bronze,total

from :  Olympics_Games.csv

* edition,edition_url,year,city,country_flag_url,country_noc,start_date,end_date,isHeld,competition_start_date,competition_end_date

 

From the above two csv files,
create a hbase data table from joining both CSV  data: olympic game medal tally with location of competition.

You have to 
* Join only necessary columns (no redundant column)
* create propoer column families by grouping related data into a family to be easy for query.

* scan the table by selecting only data with "gold" medal
Consider using scan with filter.

https://happybase.readthedocs.io/en/latest/user.html#scanning-over-rows-in-a-table


 https://hbase.apache.org/book.html#thrift


##Create new table for join 2 table

ใช้คำสั่ง 

`create "sample_data:Join_table", "edition", "country", "data", "medal" `

สร้างตารางก่อนทำการ join 2 ตารางเข้าด้วยกัน

In [ ]:
#Join only necessary columns (no redundant column)

conn = happybase.Connection(host = host,
        table_prefix = namespace,
        table_prefix_separator = ":")
conn.open()

table_1 = conn.table('Olympic_Games_Medal_Tally')
table_2 = conn.table('Olympics_Games')
join_table = conn.table('Join_table')

for key1, data1 in table_1.scan():
  for edition, data2 in table_2.scan():
  
    if(data1[b'data:edition']== edition):
      #create propoer column families by grouping related data into a family to be easy for query.
      join_table.put(key1, {
        b'edition:edition_id': data1[b'data:edition_id'],
        b'edition:edition': data2[b'data:edition'],

        b'country:city': data2[b'data:city'],
        b'country:country_noc': data2[b'data:country_noc'],

        b'data:start_date': data2[b'data:start_date'],
        b'data:end_date': data2[b'data:end_date'],
        b'data:year': data1[b'data:year'],

        b'medal:gold': data1[b'data:gold'],
        b'medal:silver': data1[b'data:silver'],
        b'medal:bronze': data1[b'data:bronze'],
        b'medal:total': data1[b'data:total'],
        })

In [ ]:
# filter only row that have gold medal
for key, data in join_table.scan():
        if(data[b'medal:gold'] != b'0') :
                print(key, data)

b'1000' {b'country:city': b'Beijing', b'country:country_noc': b'CHN', b'data:end_date': b'2008-08-24', b'data:start_date': b'2008-08-08', b'data:year': b'2008', b'edition:edition': b'2008 Summer Olympics', b'edition:edition_id': b'53', b'medal:bronze': b'20', b'medal:gold': b'7', b'medal:silver': b'16', b'medal:total': b'43'}
b'1001' {b'country:city': b'Beijing', b'country:country_noc': b'CHN', b'data:end_date': b'2008-08-24', b'data:start_date': b'2008-08-08', b'data:year': b'2008', b'edition:edition': b'2008 Summer Olympics', b'edition:edition_id': b'53', b'medal:bronze': b'4', b'medal:gold': b'7', b'medal:silver': b'5', b'medal:total': b'16'}
b'1002' {b'country:city': b'Beijing', b'country:country_noc': b'CHN', b'data:end_date': b'2008-08-24', b'data:start_date': b'2008-08-08', b'data:year': b'2008', b'edition:edition': b'2008 Summer Olympics', b'edition:edition_id': b'53', b'medal:bronze': b'11', b'medal:gold': b'7', b'medal:silver': b'4', b'medal:total': b'22'}
b'1003' {b'country: